In [5]:
import requests as r
from bs4 import BeautifulSoup as bs
import urllib
import json
import time
import unicodedata
from math import ceil

def dict_extract(key, var):
    if hasattr(var,'items'):
        for k, v in var.items():
            if k == key:
                yield v
            if isinstance(v, dict):
                for result in dict_extract(key, v):
                    yield unicodedata.normalize('NFKC',bs(result).text).strip()
            elif isinstance(v, list):
                for d in v:
                    for result in dict_extract(key, d):
                        yield unicodedata.normalize('NFKC',bs(result).text).strip()
class clarin():
    def __init__(self):
        self.url='https://www.clarin.com'
    
    
    urlc='https://login.clarin.com/comments.getComments'
    urlcp='https://login.clarin.com/comments.getComments?categoryID=Com_03&streamID=H1Y2WMTS-&includeSettings=true&threaded=true&includeStreamInfo=true&includeUserOptions=true&includeUserHighlighting=true&lang=es&ctag=comments_v2&APIKey=2_fq_ZOJSR4xNZtv2rA8DALl1Gxp7yTYMb3UdER6zerupB55mwkzh9pVBz4Blzi8SW&source=showCommentsUI&sourceData=%7B%22categoryID%22%3A%22Com_03%22%2C%22streamID%22%3A%22H1Y2WMTS-%22%7D&sdk=js_latest&authMode=cookie&pageURL=https%3A%2F%2Fwww.clarin.com%2Fpolitica%2Facuerdo-cambiemos-massismo-echar-vido-camara-diputados_0_H1Y2WMTS-.html&format=jsonp&callback=gigya.callback&context=R4169081209'
    urlp=urllib.parse.urlparse(urlcp)
    keys=urllib.parse.parse_qs(urlp.query)
 
    
    def get(self,url):
        self.url=url
        nota=r.get(url)
        sopa=bs(nota.content,features="lxml")
        ps=sopa.find('div','body-nota').findAll('p')
        st=sopa.find('div','body-nota').findAll('strong')
        self.volanta=sopa.find('p','volanta').text
        self.titulo=sopa.find('h1').text
        self.bajada=sopa.find('div','bajada').find('h2').text
        texto=list()
        for p in ps:
            if p.text == "COMENTARIOS":
                break
            texto.append(p.text)
        bolds=list()    
        for b in st:
            bolds.append(b.text)            
        self.cuerpo=' '.join(texto)
        self.bold=' '.join(bolds)
        self.bolds=bolds 
        self.get_comments()
        
    def get_comments(self):
        url=self.url
        url0='https://login.clarin.com/comments.getStreamInfo'
        keys_0={'categoryID': ['Com_03'],
         'APIKey': ['2_fq_ZOJSR4xNZtv2rA8DALl1Gxp7yTYMb3UdER6zerupB55mwkzh9pVBz4Blzi8SW'],
         'sdk': ['js_latest'],
         'authMode': ['cookie'],
         'format': ['jsonp'],
         'callback': ['gigya.callback']}

        keys_0['streamID']=url[-14:-5]
        #keys_0['pageURL']=url

        cm0=r.get(url0,params=keys_0)
        info = json.loads(cm0.text[15:-2])
        N=info['streamInfo']['threadCount']
        urlcm='https://login.clarin.com/comments.getComments'
        keys = {'categoryID': ['Com_03'],
         'includeSettings': ['true'],
         'threaded': ['true'],
         'includeStreamInfo': ['true'],
         'includeUserOptions': ['true'],
         'includeUserHighlighting': ['true'],
         'lang': ['es'],
         'ctag': ['comments_v2'],
         'APIKey': ['2_fq_ZOJSR4xNZtv2rA8DALl1Gxp7yTYMb3UdER6zerupB55mwkzh9pVBz4Blzi8SW'],
         'source': ['showCommentsUI'],
         'sdk': ['js_latest'],
         'authMode': ['cookie'],
         'format': ['jsonp'],
         'callback': ['gigya.callback']}

        keys['streamID']=url[-14:-5]
        coms=[]
        for x in range(ceil(N/10)):    
            req = json.loads(r.get(urlcm,params=keys).text[15:-2])

            #hay replies dentro de replies, busco recursivamente comentarios
            coms.extend(list(dict_extract('commentText', req)))

            #coms.extend([unicodedata.normalize('NFKC',bs(x['commentText']).text) for x in req['comments']])
            #for k in req['comments']:
            #    try: 
            #        coms.extend([unicodedata.normalize('NFKC',bs(rep['commentText']).text) for rep in k['replies']])
            #    except:
            #        pass

            req['comments'][-1]['timestamp']
            keys['start']='ts_'+str(req['comments'][-1]['timestamp'])
        self.coms=coms

def unq(url):
    urlp=urllib.parse.urlparse(url)
    keys=urllib.parse.parse_qs(urlp.query)
    return(keys)
    

In [6]:
nota9=clarin()
nota9.get('https://www.clarin.com/opinion/gobierno-cumple-ano-festejar_0_sFi-X87pB.html')

In [294]:
nota9.get_comments()

In [7]:
nota9.coms

['El gobierno cumple un ano con mucho por lo cual llorar!!!',
 'Consiguieron grandes logros, liberar a todos sus secuaces!!! DELINCUENTES!!!',
 'Que nota de mala leche!!!Me pueden decir que gobierno de TODO el mundo tiene algo que festejar este año???Clarin cada vez menos creíble!!',
 'ACÁ TENEMOS ALGO QUE FESTEJAR CADA VEZ FALTA MENOS PARA QUE SE VAYAN',
 'cuanto troll del mamerto y la borracha!!!!',
 'Creoque la yarara de recoleta es korrupta, ladrona. perversa , pero no borracha. No la calumnie.-',
 'El gran problema argentino es que entre los peores de todos los tiempos (Alberto-----Cristina) estamos nosotros en el Titanic y el iceberg ya impactó.',
 'Fernandez Renuncia,necesitamos claridad para ver con que nos enfrentamos,salite y deja de embarrar la cancha....en las próximas elecciones son porota Vs Nosotros',
 'Fernandez ya tenes la jubileta,inutil! mas los dolares que pago la porota para que le hagas de testaferro del poder....llego tu hora game over, renuncia!!!!! asi enfrenta

In [284]:
def get_comments(url):
    url0='https://login.clarin.com/comments.getStreamInfo'
    keys_0={'categoryID': ['Com_03'],
     'APIKey': ['2_fq_ZOJSR4xNZtv2rA8DALl1Gxp7yTYMb3UdER6zerupB55mwkzh9pVBz4Blzi8SW'],
     'sdk': ['js_latest'],
     'authMode': ['cookie'],
     'format': ['jsonp'],
     'callback': ['gigya.callback']}

    keys_0['streamID']=url[-14:-5]
    #keys_0['pageURL']=url

    cm0=r.get(url0,params=keys_0)
    info = json.loads(cm0.text[15:-2])
    N=info['streamInfo']['threadCount']

    keys = {'categoryID': ['Com_03'],
     'includeSettings': ['true'],
     'threaded': ['true'],
     'includeStreamInfo': ['true'],
     'includeUserOptions': ['true'],
     'includeUserHighlighting': ['true'],
     'lang': ['es'],
     'ctag': ['comments_v2'],
     'APIKey': ['2_fq_ZOJSR4xNZtv2rA8DALl1Gxp7yTYMb3UdER6zerupB55mwkzh9pVBz4Blzi8SW'],
     'source': ['showCommentsUI'],
     'sdk': ['js_latest'],
     'authMode': ['cookie'],
     'format': ['jsonp'],
     'callback': ['gigya.callback']}

    keys['streamID']=url[-14:-5]
    coms=[]
    for x in range(ceil(N/10)):    
        req = json.loads(r.get(urlcm,params=keys).text[15:-2])

        #hay replies dentro de replies, busco recursivamente comentarios
        coms.extend(list(dict_extract('commentText', req)))

        #coms.extend([unicodedata.normalize('NFKC',bs(x['commentText']).text) for x in req['comments']])
        #for k in req['comments']:
        #    try: 
        #        coms.extend([unicodedata.normalize('NFKC',bs(rep['commentText']).text) for rep in k['replies']])
        #    except:
        #        pass

        req['comments'][-1]['timestamp']
        keys['start']='ts_'+str(req['comments'][-1]['timestamp'])
    return(coms)

In [283]:
info

{'streamInfo': {'streamID': '1ttDXKDXe',
  'status': 'enabled',
  'categoryID': 'Com_03',
  'commentCount': 0,
  'approvedCommentCount': 0,
  'threadCount': 0,
  'ratingCount': 0,
  'rssURL': 'http://comments.us1.gigya.com/comments/rss/6168921/Com_03/1ttDXKDXe',
  'isUserSubscribed': False,
  'moderationMode': 'inherit',
  'moderationModes': {'text': 'inherit',
   'image': 'inherit',
   'video': 'inherit',
   'url': 'inherit',
   'other': 'inherit'}},
 'statusCode': 200,
 'errorCode': 0,
 'statusReason': 'OK',
 'callId': '916b7ecd5aec4c7a97e860cc4b78bb5c',
 'time': '2020-12-08T12:58:21.321Z'}

In [281]:
len(coms)

0

In [278]:
coms

['Macri es mufa!!',
 'Los años bisiestos generalmente son muy tumultuosos...',
 'El "como fue para vos el 2016" no deja encuestar ni ver resultados, te dirige a una foto de un planeta, no hagan kirnereadas donde todo es falso.',
 'ARGENTINA ES LA EXCEPCION, con tantos años de peronismo y kirchnrismo nos acostumbraron a las desgracias rutinarias, este fue de gracia para nostoros a pesar que el peronchismo hizo lo posible para interrumpir, aobstaculizar, inhabilitas, enturbiecer, etc tuvimos un año de gracia, brindo por 100 sin peronistas',
 'Bueno, parece que te olvidás de un sr. que presidía el país allá por el 2001, por diciembre, te acordás? Mucha gente no lo va a olvidar nunca. Ah, no era peroncho, no, estoy seguro que no lo era.',
 'uno....un señor....y todos los otros señores? de donde salieron?',
 'Para mi el mejor año. Un año sin KAKAS en el poder. Más no puedo pedír',
 'han pasado cosas peores en otros años, desgracias siempre las hay. dejen de ser tan pesimistas.',
 'fue desgr